In [ ]:
from itertools import groupby
def rle(data):
  x = [len(list(y)) for x, y in groupby(data) if x==True]
  if x==[]:
    return 0
  return max(x)
#Obliczenie różnic
dfr=df.copy()
nr_pacjenta=dfr["PACJENT_NR"]
dfr2=dfr[yeslist].groupby('PACJENT_NR').diff()
dfr3=pd.concat([nr_pacjenta, dfr2], axis=1)
dfr_mdiff=dfr3.groupby('PACJENT_NR').mean().add_suffix('_mdiff')
#dfr_mdiff

df_mean = df[yeslist].groupby('PACJENT_NR').mean().add_suffix('_mean')
df_min = df[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max = df[yeslist].groupby('PACJENT_NR').max().add_suffix('max')
df_med = df[yeslist].groupby('PACJENT_NR').median().add_suffix('median')
df_size = df[['PACJENT_NR', 'BADANIE_NR']].groupby('PACJENT_NR').count().add_suffix('_size')
df_pom = df[yeslist].groupby('PACJENT_NR').std().add_suffix('_std')
df_sum = df[yeslist].groupby('PACJENT_NR').sum().add_suffix('_sum')

df_rzad_pom_ups=pd.concat([df_imp["PACJENT_NR"],df_imp[["BETET", "IONH", "O2SATTET", "CO2TET"]]>1], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_ups=df_rzad_pom_ups.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_ups')

df_rzad_pom_downs=pd.concat([df_imp["PACJENT_NR"],df_imp[["BETET", "IONH", "O2SATTET", "CO2TET"]]<0], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_downs=df_rzad_pom_downs.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_downs')

df_zgon = df_imp[['PACJENT_NR', 'ZGON']].groupby('PACJENT_NR').first()

In [ ]:
#Zliczanie wyników za niskich
df_no_drops = df_imp.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_drops[col] = np.where(df_imp[col] < 0, 1, 0) #1-za niski, 0-nie za niski

#Zliczanie wyników za wysokich
df_no_ups = df_imp.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_ups[col] = np.where(df_imp[col] > 1, 1, 0) #1-za wysoki, 0-nie za wysoki

#Zliczanie wyników poza normą drop-poniżej up-powyżej
df_no_dropup = df_imp.copy()
for col in yeslist:
    if col != "PACJENT_NR": #dla innych kolumn niż numer pacjenta
        df_no_dropup[col] = df_no_ups[col] + df_no_drops[col]

#Dodanie kolumny nOK zliczającej, ile wyników pacjent miał ok
df_no_dropup["nOK"] = 0
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_dropup["nOK"] += df_no_dropup[col]

#Dodanie kolumny sOK, która przyjmuje wartość 1, gdy wszystkie wyniki były ok
df_no_dropup["sOK"] = np.where(df_no_dropup["nOK"] == 0, 1, 0)

#wypisz tablicę wszystkich wyników
#print(df_no_dropup)


In [ ]:
#agregacja
df_no_drops = df_no_drops[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_drops')
df_no_ups = df_no_ups[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_up')
df_no_dropup = df_no_dropup[["PACJENT_NR","BETET", "IONH", "O2SATTET", "CO2TET", "nOK", "sOK"]].groupby('PACJENT_NR').sum().add_suffix('_no_dropup')

#"O2TET", "HCO3STTE", "HCO3ACTE"

#łącznie
df_list = [df_size,df_zgon, df_min, df_max, df_no_drops, df_no_ups, df_no_dropup, dfr_mdiff, df_rzad_ups, df_rzad_downs]
#'df_mean, df_med, df_pom, df_sum'
df_all = pd.concat(df_list, axis=1)


#usuwanie kolumn
df_all.drop(columns=["O2SATTET_no_up", "O2SATTET_no_dropup", "O2SATTET_rzad_ups" ,  "O2SATTETmax"], inplace=True) #"O2TETmax" ,  "O2SATTETmax"

# df_all.to_csv("pacjenci_statystyki_gazometria.csv")
df_all.columns
df_all = df_all[df_all["BADANIE_NR_size"] > 6]
df_all["ZGON"].replace({0: "recovered", 1: "deceased"}, inplace=True)
df2_dead = df_all[df_all["ZGON"]=="deceased"].copy()
df2_life = df_all[df_all["ZGON"]=="recovered"].copy()

df2_dead.drop(columns=["ZGON"], inplace=True)
df2_life.drop(columns=["ZGON"], inplace=True)
df2_dead.columns
# plt.figure(figsize=(10,5))

In [ ]:

#drzewo decyzyjne z walidacją krzyżową i zbalansowaniem klas przy podziale na treningowe i testowe
X=df_all[nazwy_sign]
y=df_all["ZGON"]
model = tree.DecisionTreeClassifier(random_state=5, max_depth=3)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=18) #5

s=list()
stren=list()
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  s.append(np.round(model.score(X.iloc[test_index], y.iloc[test_index]),4))
  stren.append(np.round(model.score(X.iloc[train_index], y.iloc[train_index]),4))
  # confusion = confusion_matrix(y.iloc[test_index], model.predict(X.iloc[test_index]))
  #print(confusion)
  
print("Wyniki uczenia: ", stren)
print("Dokładność uczenia: ", np.mean(stren))
print()
print("Wyniki kroswalidacji: ", s)
print("Dokładność drzewa: ", np.mean(s))
model.fit(X, y)
wartosci=model.feature_importances_
print(X.columns)
kategorie=['minimum BETET \n measurement', 'minimum [H$^\plus$] \n (maximum pH)', 'maximum [H$^\plus$] (minimum pH)', 'maximum pCO$_2$', 'BETET below the norm [number of cases]',
           '[H$^\plus$] below the norm \n (pH above the norm [number of cases])', 'pO$_2$ saturation below the norm \n [number of cases]', 'pCO$_2$ below the norm [number of cases]',
           'BETET above the norm [number of cases]',
           '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases]', 'average change in \n ion H concentration', 'average change in \n O$_2$ saturation', 'average change in pCO$_2$',
           'BETET above the norm [number of cases in a row]', '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases in a row]', 'BETET below the norm [number of cases in a row]',
           '[H$^\plus$] below the norm \n (pH above the norm) \n [number of cases in a row]', 'O$_2$ saturation above the norm \n [number of cases in a row]']
# kategorie=X.columns

In [ ]:
# Tworzenie wykresu słupkowego
niezerowe_wartosci = [x for x in wartosci if x != 0]
niezerowe_kategorie = [kategorie[i] for i, x in enumerate(wartosci) if x != 0]

#sortowanie
sorted_niezerowe_wartosci, sorted_niezerowe_kategorie = zip(*sorted(zip(niezerowe_wartosci, niezerowe_kategorie), reverse=False))

# Tworzenie wykresu słupkowego z niezerowymi wartościami
fig, ax = plt.subplots(figsize=(7,5))
plt.barh(sorted_niezerowe_kategorie, sorted_niezerowe_wartosci)

plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
ax.yaxis.set_label_coords(-0.5, 0.5)
plt.show()

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=[ "ZGON - TAK","ZGON - NIE"],
                           filled=False, rounded=True)

graph = graphviz.Source(dot_data)

# Zapisz obrazek do pliku PNG
# graph.format = 'png'
# graph.render('drzewo_gazometria_cale_dane')
graph

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=["ZGON - NIE", "ZGON - TAK"],
                           filled=False, rounded=True)
  graph = graphviz.Source(dot_data)
  graph.format = 'png'
  graph.render(f'drzewo_gazometria_{i}')


In [ ]:

#srednia wynikow min i max
yeslist = ["PACJENT_NR","BETET", "IONH", "O2SATTET",  "CO2TET", "HCO3STTE", "HCO3ACTE", "O2TET", "PHTET"]
df_min2 = df6[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max2 = df6[yeslist].groupby('PACJENT_NR').max().add_suffix('max')

df_list2 = [df_zgon, df_size, df_min2, df_max2 ]
df_min_max = pd.concat(df_list2, axis=1)

#df_min_max['PHmin'] = -np.log10(df_min_max['IONHmin'])
#df_min_max['PHmax'] = -np.log10(df_min_max['IONHmax'])

df_min_max.head()

srednie = df_min_max.groupby('ZGON').mean()
print(srednie)
# df_size = df0[['PACJENT_NR', 'BADANIE_NR']].groupby('PACJENT_NR').count().add_suffix('_size')
# pacjenci_wiecej_niz_6_badan = df_size[df_size['BADANIE_NR_size'] > 6].index

# df_filtered = df0[df0['PACJENT_NR'].isin(pacjenci_wiecej_niz_6_badan)]
# df_filtered1 = df_filtered[df_filtered['BADANIE_NR'] == 1]
# df_filtered1["ZGON"] = np.where(df_filtered1["ZGON"] == "NIE", 0, 1)
# df_filtered1.head()

# df_mean_1 = df_filtered1[yeslist].groupby('ZGON').mean().add_suffix('_mean')
# print(df_mean_1)